In [ ]:
import matplotlib
matplotlib.use('Agg') # disable interactive matplotlib to save RAM

import glob 
import matplotlib.pyplot as plt
import numpy as np
import os 
import pandas as pd
import shutil
import spikeinterface.core as sc
import spikeinterface.extractors as se
import sys

from tqdm.auto import tqdm

sys.path.extend(['src', f'src{os.sep}brpylib', f'src{os.sep}load_intan_rhd_format'])

from brpylib import NsxFile
from load_intan_rhd_format import read_data as read_rhd_data
from utils import memory_limit, n_s_per_min, blackrock_channel_indices, intan_channel_indices, create_probe, plot_traces

data_root = 'data'
si_folder = f'{data_root}{os.sep}spikeinterface-0_98_2'

# Package Long-Term Recordings (assuming blackrock .nsx files)

Read raw .nsx files. Create Spikeinterface recordings objects with the preset multi-shank probe.

**Assuming file structure: YYYYMMDD/X_Y/recording.nsx**, where X=wafer laber and Y=device label.

In [ ]:
# Process blackrock recording nsx files. Each contains entire recording session and will be saved together.
nsx = 'ns4'
n_shank, n_channel_per_shank = blackrock_channel_indices.shape
n_channel = blackrock_channel_indices.size
longterm_folder = f'{data_root}{os.sep}raw{os.sep}LongTerm{os.sep}'
recording_paths = glob.glob(f'{longterm_folder}**{os.sep}*.{nsx}', recursive=True)

def read_nsx(recording_path, trace_scale=4):
    nsxfile = NsxFile(recording_path)
    raw_data = nsxfile.getdata()
    sampling_frequency = raw_data['samp_per_s']
    raw_traces = raw_data['data'][0][:n_channel] / trace_scale
    nsxfile.close()
    return raw_traces, sampling_frequency
    
for recording_path in (pbar := tqdm(sorted(recording_paths))):
    date, mouse, recording_name = recording_path[len(longterm_folder):].split(os.sep)
    pbar.set_description(f'{mouse} -> {date} -> {recording_name}')
    recording_name = recording_name.split('.')[0]
    recording_si_path = f'{si_folder}{os.sep}LongTerm-{nsx}{os.sep}{mouse}{os.sep}{date}{os.sep}{recording_name}{os.sep}raw'
    recording_trace_plot_file = f'{recording_si_path}{os.sep}traces.png'
    
    if not os.path.isfile(f'{recording_si_path}{os.sep}binary.json'):
        pbar.set_description(f'{mouse} -> {date} -> {recording_name}')
        shutil.rmtree(recording_si_path, ignore_errors=True)
        raw_traces, sampling_frequency = read_nsx(recording_path)
        
        recording = se.NumpyRecording(traces_list=raw_traces.T, sampling_frequency=sampling_frequency)
        multi_shank_probe, fig = create_probe(blackrock_channel_indices)
        recording.set_probe(multi_shank_probe, in_place=True)
        recording.save(folder=recording_si_path, memory=memory_limit)
        fig.savefig(f'{recording_si_path}{os.sep}probe.png')
        plt.close()
    
    if not os.path.isfile(recording_trace_plot_file):
        recording = sc.load_extractor(recording_si_path)
        plot_traces(recording, blackrock_channel_indices, title=f'{mouse} -> {date} -> {recording_name}', savepath=recording_trace_plot_file)

# Package Behavioral Recordings (assuming intan .rhd files)

Read raw .rhd files. Create Spikeinterface recordings objects with the preset multi-shank probe.

**Assuming file structure: YYYYMMDD/X_Y/session_N/recording_[times].rhd**, where X=wafer laber, Y=device label and N=recording session label, and times are continuous 1-minute marks for recordings.

In [ ]:
# Process intan recording rhd files. Each session is described by a folder containing continuous 1-minute rhd recording files.
n_shank, n_channel_per_shank = intan_channel_indices.shape
n_channel = intan_channel_indices.size
behavior_folder = f'{data_root}{os.sep}raw{os.sep}Behavior'
session_paths = glob.glob(f'{behavior_folder}{os.sep}**{os.sep}**{os.sep}**')

def read_rhd(recording_path, blank_start=7, blank_end=9):
    raw_data = read_rhd_data(recording_path)
    sampling_frequency = raw_data['frequency_parameters']['amplifier_sample_rate']
    raw_traces = np.vstack([raw_data['amplifier_data'][:blank_start], raw_data['amplifier_data'][blank_end:]])
    return raw_traces, sampling_frequency



for session_path in (pbar := tqdm(session_paths)):
    date, mouse, session_name = session_path[len(f'{behavior_folder}{os.sep}'):].split(os.sep)
    pbar.set_description(f'{mouse} -> {date} -> {session_name}')
    session_si_path = f'{si_folder}{os.sep}Behavior{os.sep}{mouse}{os.sep}{date}{os.sep}{session_name}{os.sep}raw'
    session_trace_plot_file = f'{session_si_path}{os.sep}traces.png'

    if not os.path.isfile(f'{session_si_path}{os.sep}files.csv'):
        session_traces, session_recording_paths = [], []
        cumulative_samples = 0
        for recording_path in sorted(glob.glob(f'{session_path}{os.sep}*.rhd')):
            recording_name = recording_path.split(os.sep)[-1].split('.')[0]
            pbar.set_description(f'{session_name} -> {recording_name}')

            try:
                raw_traces, sampling_frequency = read_rhd(recording_path)
                duration = raw_traces.shape[1] // sampling_frequency 
                if duration != n_s_per_min:
                    print(f'[duration {duration:0.0f}s] discarding {session_name}{os.sep}{recording_name}')
                    continue
            except:
                print(f'[corrupted] discarding {session_name}{os.sep}{recording_name}')
                continue

            session_traces.append(raw_traces)
            session_recording_paths.append({
                'mouse': mouse, 
                'date': date,
                'session': session_name,
                'file': recording_path,
                'file_start': cumulative_samples,
                'sampling_frequency': sampling_frequency,
                'file_length': raw_traces.shape[1],
            })
            cumulative_samples += raw_traces.shape[1]
        session_traces = np.hstack(np.array(session_traces))

        recording = se.NumpyRecording(traces_list=session_traces.T, sampling_frequency=sampling_frequency)
        multi_shank_probe, fig = create_probe(intan_channel_indices)
        recording.set_probe(multi_shank_probe, in_place=True)
        recording.save(folder=session_si_path, memory=memory_limit)
        fig.savefig(f'{session_si_path}{os.sep}probe.png')
        plt.close()

        session_recording_paths = pd.json_normalize(session_recording_paths)
        session_recording_paths.to_csv(f'{session_si_path}{os.sep}files.csv', index=False)

    if not os.path.isfile(session_trace_plot_file):
        recording = sc.load_extractor(session_si_path)
        plot_traces(recording, intan_channel_indices, title=f'{mouse} -> {date} -> {session_name}', savepath=session_trace_plot_file)